# Toy model: quadratic function, ressembling weak-lensing power spectrum

In [ ]:
from os import environ
import numpy as np
import pylab as plt

In [ ]:
%matplotlib inline

### Weak-lensing power spectrum

In [ ]:
basedir = '{}/astro/papers/cs_review/figures/dlogP_dlogp'.format(environ['HOME'])

In [ ]:
def read_Cell(path, ell_min, ell_max):
    """Read P_kappa file and return log_10(ell), log_10(C_ell) within ell-range
    """
    
    pk = np.loadtxt(path)
    ell  = pk[:,0]
    Cell = pk[:,1] / (ell * (ell+1) / (2 * np.pi))

    ran     = (ell>=ell_min) & (ell<=ell_max)
    
    return np.log10(ell[ran]), np.log10(Cell[ran])

#### Fitting range

In [ ]:
ell_min = 10
ell_max = 1e4

#### Fiducial cosmological parameters

In [ ]:
Om_fid = 0.306
s8_fid = 0.827

#### Plot standard power spectrum

In [ ]:
logell, logCell = read_Cell('{}/P_kappa'.format(basedir), ell_min, ell_max)

In [ ]:
p = plt.plot(logell, logCell)
tx = plt.xlabel('$\ell$')
ty = plt.ylabel('$C_\ell$')

## Fitting functions
We define a quadratic function $q(u) = c + a (u - u_0)^2$, which we will fit to the (ten-)logarithm of the weak-lensing power spectrum $C_\ell$. We will then vary two parameters, that mimic cosmological parameter as follows:

1. A tilt parameter `tilt`, corresponding to the matter density $\Omega_{\rm m}$. This is linearly related to the shift $u_0$.

2. An amplitude parameter `ampl`, corresponding to the power-spectrum amplitude $\sigma_8$. The logarithm of that parameter is linearly related to the offset $c$.

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def add(ampl):
    """Return the additive constant of the quadratic function
       from the amplitude fitting parameter
       (mimics power-spectrum normalisation s8)
    """
    
    # This provides a best-fit amp=0.827, but the 10% increased
    # spectrum (0.9097) gives a best-fit of 0.925
    # Changing the prefactor of amp or lg(amp) does not help...
    
    c = np.log10(ampl)*2 - 6.11568527 + 0.1649
    
    return c

In [ ]:
def shift(tilt):
    """Return the shift parameter of the quadratic function
       from the tilt parameter (mimics matter density)
    """
    
    u0 = tilt * 1.85132114 / 0.306
    
    return u0

In [ ]:
def quadratic(u, *params):
    """Used to fit quadratic function varying all three parameters
    """
    
    (ampl, tilt, a) = np.array(params)
    c  = add(ampl)
    u0 = shift(tilt)
    
    return c + a * (u - u0)**2

In [ ]:
def quadratic_c(u, *params):
    """Used to fit quadratic function varying constant coefficient only
    """
    (ampl) = np.array(params)
    c  = add(ampl)
    u0 = shift(tilt_glob)
    
    return c + a_glob * (u - u0)**2

In [ ]:
def quadratic_u0(u, *params):
    """Used to fit quadratic function varying shift parameter only
    """
    (tilt) = np.array(params)
    c = add(ampl_glob)
    u0 = shift(tilt)
    
    return c + a_glob * (u - u0)**2

## Fitting

### Fit standard power spectrum

In [ ]:
p0 = [1, 0, 0]
popt, pcov = curve_fit(quadratic, logell, logCell + logell, p0)

Best-fit parameter (ampl, tilt, $u_0$)

In [ ]:
popt

In [ ]:
plot_1 = plt.plot(logell, logCell + logell, 'r.', label='Data')
plt.plot(logell, quadratic(logell, *popt), 'r-', label='quadratic fit')
plt.legend()
xl_1   = plt.xlabel('$\lg \ell$')
yl_1   = plt.ylabel('$\lg [\ell \, C(\ell)]$')

### Fit standard power spectrum with higher amplitude (small-scale normalization $\sigma_8$)

The normalisation is increased by 10%, thus $\sigma_8^+ = 0.9097$.

#### Fit by varying all coefficients

In [ ]:
logell_ps, logCell_ps = read_Cell('{}/normalization+dp/P_kappa'.format(basedir), ell_min, ell_max)

In [ ]:
p0 = [1, 0, 0]
popt_ps, pcov_ps = curve_fit(quadratic, logell_ps, logCell_ps + logell_ps, p0)

In [ ]:
popt_ps

#### Fit by only varying amplitude

In [ ]:
tilt_glob, a_glob  = popt[1], popt[2]
p0  = [1]
popt_ps_A, pcov_ps_A = curve_fit(quadratic_c, logell_ps, logCell_ps + logell_ps, p0)

Best-fit ampl

In [ ]:
popt_ps_A

### Fit standard power spectrum with higher matter density $\Omega_{\rm m}$ (tilt and amplitude parameter)

Increase by 10% gives $\Omega_{\rm m}^+ = 0.3366$.

#### Fit by varying all parameters

In [ ]:
logell_pO, logCell_pO = read_Cell('{}/Omega_m+dp/P_kappa'.format(basedir), ell_min, ell_max)

In [ ]:
p0 = [1, 0, 0]
popt_pO, pcov_pO = curve_fit(quadratic, logell_pO, logCell_pO + logell_pO, p0)

In [ ]:
popt_pO

#### Fit by varying only shift parameter

In [ ]:
ampl_glob, a_glob  = popt[0], popt[2]
p0  = [0]
popt_pO_x0, pcov_pO_x0 = curve_fit(quadratic_u0, logell_pO, logCell_pO + logell_pO, p0)

In [ ]:
popt_pO_x0

In [ ]:
d1, = plt.plot(logell, logCell + logell, 'r.')
d2, = plt.plot(logell_ps, logCell_ps + logell, 'b.')
d3, = plt.plot(logell_pO, logCell_pO + logell, 'g.')

f1, = plt.plot(logell, quadratic(logell, *popt), 'r-')
f2, = plt.plot(logell_ps, quadratic_c(logell_ps, *popt_ps_A), 'b:')
f3, = plt.plot(logell_pO, quadratic_u0(logell_pO, *popt_pO_x0), 'g:')

plt.plot(logell_ps, quadratic(logell_ps, *popt_ps), 'b--')
plt.plot(logell_pO, quadratic(logell_pO, *popt_pO), 'g--')

plot_data  = [d1, d2, d3]
label_data = ['Original data', 'Larger amplitude', 'Larger tilt']
leg_data   = plt.legend(plot_data, label_data, loc=1)

plot_fit  = [f1, f2, f3]
label_fit = ['3-parameter fit', 'Amplitude fit', 'Tilt fit']
leg_fit   = plt.legend(plot_fit, label_fit, loc=3)

plt.gca().add_artist(leg_data)

text_x = plt.xlabel('$\log \ell$')
text_y = plt.ylabel('$\log(\ell C_\ell)$')

In [ ]:
msize = 8
d1, = plt.plot(logell, logCell + logell, 'r.', markersize=msize)
d2, = plt.plot(logell_ps, logCell_ps + logell, 'bs', markersize=msize-5)
d3, = plt.plot(logell_pO, logCell_pO + logell, 'gD', markersize=msize-4)

f1, = plt.plot(logell, quadratic(logell, *popt), 'r-')
f2, = plt.plot(logell_ps, quadratic_c(logell_ps, *popt_ps_A), 'b--')
f3, = plt.plot(logell_pO, quadratic_u0(logell_pO, *popt_pO_x0), 'g:')

#plt.plot(logell_ps, quadratic(logell_ps, *popt_ps), 'b--')
#plt.plot(logell_pO, quadratic(logell_pO, *popt_pO), 'g--')

plot_data  = [d1, d2, d3]
label_data = ['Original data', 'Larger amplitude', 'Larger tilt']
leg_data   = plt.legend(plot_data, label_data, loc=1)

plot_fit  = [f1, f2, f3]
label_fit = ['3-param. fit to orig. data', 'Free amplitude fit', 'Free tilt fit']
leg_fit   = plt.legend(plot_fit, label_fit, loc=3)

plt.gca().add_artist(leg_data)

text_x = plt.xlabel('$\log \ell$')
text_y = plt.ylabel('$\log(\ell C_\ell)$')

### Covariance

#### Parameter and constants

In [ ]:
from astropy import units
sigma_eps  = 0.22
nbar       = 30
nbar_amin2 = units.Unit('{}/arcmin**2'.format(nbar))
nbar_rad2  = nbar_amin2.to('1/rad**2')
f_sky           = 0.3636

#### Total power spectrum, including shot-/shape-noise term

In [ ]:
C_ell_tot = 10**logCell + sigma_eps**2 / (2 * nbar_rad2)

In [ ]:
ell = 10**logell

#### Bin width $\Delta_\ell$ when $\ell$ are logarithmic, and thus $\Delta \ln \ell = \Delta \ell / \ell$ = const

In [ ]:
Delta_ln_ell = np.diff(ell) / (ell[:-1]/2 + ell[1:]/2)
Delta_ln_ell = np.append(Delta_ln_ell, Delta_ln_ell[-1])
Delta_ell = Delta_ln_ell * ell
Delta_ln_ell_bar = Delta_ln_ell.mean()

#### Gaussian covariance

In [ ]:
D = 1.0 / (f_sky * (2.0 * ell + 1) * Delta_ln_ell_bar * ell) * C_ell_tot**2
D

#### Approximation of $D$

In [ ]:
A = 1/(2 * f_sky * Delta_ln_ell_bar)
B = sigma_eps**2 / (2 * nbar_rad2)
D2 = A / ell**2 * (10**quadratic(logell, *popt) / ell + B)**2

#### Read from file, created by ABC run

In [ ]:
Dinv = np.loadtxt('cov_true_inv_diag.txt')

In [ ]:
d1, = plt.plot(logell, np.log10(D))
d2, = plt.plot(logell, np.log10(D2))
d3, = plt.plot(logell, np.log10(1/Dinv))

plot_data  = [d1, d2, d3]
label_data = ['Original cov', 'Approx. cov', 'ABC run cov']
leg_data   = plt.legend(plot_data, label_data, loc=1)

leg = plt.gca().add_artist(leg_data)

In [ ]:
D2

In [ ]:
popt

In [ ]:
B = sigma_eps**2 / (2 * nbar_rad2) 

In [ ]:
5000. / (np.pi*4 * 180**2 / np.pi**2)